[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1U9fI5iPjMGqSfs0YSMzpDa2GKLQYSf4x?usp=sharing)


In [ ]:
# !pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset('AnyaSchen/image2poetry_ru')

Found cached dataset parquet (/home/revolt/.cache/huggingface/datasets/AnyaSchen___parquet/AnyaSchen--image2poetry_ru-bd53c8b353e828ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00, 382.06it/s]


In [ ]:
import pickle
with open('../generated/image2poetry/vit_rugpt2_ft.pkl', 'rb') as f:
  data_dict = pickle.load(f)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter

def calculate_distinct_n(words, n):
    # tokenize the text into words, assuming whitespace separated words
    # words = text.split()
    
    # calculate n-grams
    n_grams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    
    # count distinct n-grams
    distinct_n_grams = len(set(n_grams))
    
    # count total n-grams
    total_n_grams = len(n_grams)
    
    # return ratio of distinct n-grams to total n-grams
    return distinct_n_grams / total_n_grams if total_n_grams != 0 else 0

In [ ]:
metrics = []
for poet in set(dataset['train']["author"]):
  mayak_data = dataset.filter(lambda example, indice: example['author']==poet, with_indices=True)
  print(poet)
  reference_codes_tokenized = [code.split() for code in mayak_data['train']['poetry']]
  generated_codes_tokenized = [code.split() for code in data_dict[poet]]

  # Calculate BLEU scores
  bleu_scores = [sentence_bleu([reference], generated, weights=(0,0, 1, 0), smoothing_function=SmoothingFunction().method1) 
                for reference, generated in zip(reference_codes_tokenized, generated_codes_tokenized)]
  average_bleu_score = sum(bleu_scores) / len(bleu_scores)
  print(f'Average BLEU score: {average_bleu_score}')

  # Calculate distinct-n scores
  distinct_n_scores = [calculate_distinct_n(code, n=3) for code in generated_codes_tokenized]  # Here, n=3
  average_distinct_n_score = sum(distinct_n_scores) / len(distinct_n_scores)
  print(f'Average Distinct-2 score: {average_distinct_n_score}')
  metrics.append([poet, average_bleu_score, average_distinct_n_score])

Loading cached processed dataset at /home/revolt/.cache/huggingface/datasets/AnyaSchen___parquet/AnyaSchen--image2poetry_ru-bd53c8b353e828ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cd7e63f6d721a8e4.arrow


Маяковский


Loading cached processed dataset at /home/revolt/.cache/huggingface/datasets/AnyaSchen___parquet/AnyaSchen--image2poetry_ru-bd53c8b353e828ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-6c42722919799f03.arrow


Average BLEU score: 0.34667270552819446
Average Distinct-2 score: 0.9725527274614757
Блок


Loading cached processed dataset at /home/revolt/.cache/huggingface/datasets/AnyaSchen___parquet/AnyaSchen--image2poetry_ru-bd53c8b353e828ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f4a82f1f6cbc5605.arrow


Average BLEU score: 0.2482589634912962
Average Distinct-2 score: 0.96984013592479
Есенин
Average BLEU score: 0.5606370571897304
Average Distinct-2 score: 0.9782553795742579


Тютчев
Average BLEU score: 0.30301646490728834
Average Distinct-2 score: 0.9779230590544945


Пушкин
Average BLEU score: 0.3120432976279029
Average Distinct-2 score: 0.9882043421808078
